In [1]:
import pandas as pd
import numpy as np
import torch
from data_provider.data_factory import Data_Provider
import yaml, json
from utils.tools import dotdict

In [2]:
# load json
with open('checkpoints/03-11-0936_FITS_solar_day_ahead/args.json', 'r') as f:
    args = json.load(f)
args = dotdict(args)

In [3]:
# load yaml
with open('./data_configs/fullsolar_hetero.yaml') as f:
    data_config = yaml.load(f, Loader=yaml.FullLoader)
data_config = dotdict(data_config)
args.data_config = data_config

In [4]:
a=Data_Provider(args, buffer=(not args.disable_buffer))

[ info ] Successfully load the dynamic data from fast_general_weather_forecast_????.json


In [5]:
set = a.get_val('set')

Loading val datasets:  11%|█         | 1/9 [00:00<00:03,  2.31it/s]

Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_314106.parquet to buffer
Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_319086.parquet to buffer


Loading val datasets:  33%|███▎      | 3/9 [00:00<00:01,  4.19it/s]

Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_164440.parquet to buffer
Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_355827.parquet to buffer


Loading val datasets:  56%|█████▌    | 5/9 [00:01<00:00,  4.29it/s]

...

Loading val datasets:  67%|██████▋   | 6/9 [00:01<00:00,  4.73it/s]

Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_332785.parquet to buffer
Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_577650.parquet to buffer


Loading val datasets:  89%|████████▉ | 8/9 [00:01<00:00,  5.15it/s]

Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_551172.parquet to buffer
Add data /data/Blob_WestJP/v-zhijianxu/Benchmark/fullsolar/impute_data/id_570079.parquet to buffer


Loading val datasets: 100%|██████████| 9/9 [00:01<00:00,  4.58it/s]


In [6]:
set.keys()

dict_keys(['314106', '319086', '164440', '355827', '331901', '332785', '577650', '551172', '570079'])

In [7]:
seta = set['314106']
len(seta)

8569

In [8]:
seta[2000]

(array([[-0.54947427],
        [-0.54947427],
        [-0.53816769],
        [-0.51134742],
        [-0.5184469 ],
        [-0.46875051],
        [-0.47427233],
        [-0.45639215],
        [-0.5184469 ],
        [-0.54868544],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54106007],
        [-0.42930894],
        [-0.1687316 ],
        [-0.10746569],
        [ 0.15968525],
        [ 0.06081837],
        [ 0.12024367],
        [ 0.17414716],
        [-0.0577693 ],
        [-0.4926784 ],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0.54947427],
        [-0

In [9]:
seta[2000][3]

array([20210401080000, 20210401090000, 20210401100000, 20210401110000,
       20210401120000, 20210401130000, 20210401140000, 20210401150000,
       20210401160000, 20210401170000, 20210401180000, 20210401190000,
       20210401200000, 20210401210000, 20210401220000, 20210401230000,
       20210402000000, 20210402010000, 20210402020000, 20210402030000,
       20210402040000, 20210402050000, 20210402060000, 20210402070000])

## try with openai API

In [ ]:
# ts_x, ts_y, tm_x, tm_y, (dy_tm_x, general, channel, [dy_x]), (dy_tm_y, general, channel, [dy_y])

In [10]:
import openai

In [11]:
openai.base_url = 'http://localhost:8021/v1/'
openai.api_key = 'EMPTY'

In [30]:
system_prompt = "You are a professonal data analyst. You can do the forecasting by considering the historical reading time series along with weather condition and make the prediction for the next 24 hours. " + seta[2000][4][1] + "Please directly do the forecasting without any further external tools such as coding. You can consider the periodicity, trend, and seasonality of the data. Then consider the affect of weather condition to further refine your prediction. "


In [18]:
x_ts = seta[2000][0].squeeze().tolist()
x_timestamp = seta[2000][2].tolist()
x_table = [(x_timestamp[i], x_ts[i]) for i in range(len(x_ts))]

x_dy = seta[2000][4][-1]
x_dy_timestamp = seta[2000][4][0]
x_dy_table = [(x_dy_timestamp[i], x_dy[i]) for i in range(len(x_dy))]

channel_info = seta[2000][4][2]
channel_info

'The solar panel is located at Calgary Fire Hall Headquarters.'

In [17]:
y_ts = seta[2000][1].squeeze().tolist()
y_timestamp = seta[2000][3].tolist()

y_dy = seta[2000][5][-1]
y_dy_timestamp = seta[2000][5][0]
y_dy_table = [(y_dy_timestamp[i], y_dy[i]) for i in range(len(y_dy))]
y_dy_table

[('20210401000000',
  '{"daily":"It\'s going to be a chilly April day with a mix of passing clouds and broken clouds. Expect a temperature trend of warming up in the morning and cooling down in the evening. There might be a sudden shower in the late afternoon. Overall, it\'s a great day to enjoy the outdoors with a light jacket.","Early Morning":"Early morning will be quite chilly with a temperature around 2-3\\u00b0C. Expect passing clouds with a gentle breeze. It\'s a great time to get some fresh air and enjoy the morning.","Morning":"Morning will see a slight warming up with temperatures reaching around 10-12\\u00b0C. The sky will be partly sunny with a gentle breeze. It\'s a perfect time to get out and about.","Afternoon":"Afternoon will be quite windy with gusts reaching up to 50 km\\/h. Temperatures will be around 14-16\\u00b0C. Expect passing clouds with a chance of sudden showers. It\'s a good idea to carry an umbrella.","Evening":"Evening will see a cooling down with temperatu

In [ ]:
user_prompt = f'''

We now have the data from {channel_info}. The historical data is as follows:

{x_table}

The historical weather condition is as follows:

{x_dy_table}

We now have the weather forecasting for the following days:

{y_dy_table}

Please make the prediction for the next 24 hours.

The timestamp for forecasting is as follows:

{y_timestamp}

Please format it in json format in [datastamp, value] pair, as below:

```json

[[<timestamp>, <predicted_value>],
 [<timestamp>, <predicted_value>],
 ...]

```
'''

In [38]:
messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt}
]

In [39]:
result = openai.chat.completions.create(
                    model='local-model',
                    messages=messages,
                    temperature=0.2,
                    )
aaa= result.choices[0].message.content
aaa

' \n\nTo make the prediction for the next 24 hours, I will consider the historical reading time series along with weather condition.\n\nFirst, I will analyze the historical data to identify any patterns or trends. The data shows a general decreasing trend over time, with some fluctuations. There are also some periodic patterns, such as a daily cycle with a peak in the morning and a trough in the evening.\n\nNext, I will consider the weather condition data. The weather forecast for the next 24 hours indicates a mix of passing clouds and broken clouds, with a temperature trend of warming up in the morning and cooling down in the evening. There might be a sudden shower in the late afternoon.\n\nTaking into account the historical data and weather condition, I will make the prediction for the next 24 hours.\n\nHere is the prediction in JSON format:\n\n```json\n[[20210401080000, 0.1234],\n [20210401090000, 0.2345],\n [20210401100000, 0.3456],\n [20210401110000, 0.4567],\n [20210401120000, 0.